Hi! Let's analyze the results of the experiment you just ran. To recap:

1. You just ran something similar to:

    `python benchmark.py --multirun ranker="glob(*)" +callbacks.to_sql.url="sqlite:///$HOME/results.sqlite"`
2. There now should exist a `.sqlite` file at this path: `$HOME/results.sqlite`:

    ```
    $ ls -al $HOME/results.sqlite
    -rw-r--r-- 1 vscode vscode 20480 Sep 21 08:16 /home/vscode/results.sqlite
    ```

Let's now analyze the results! 📈

First, we will install `plotly-express`, so we can make nice plots later.

In [1]:
%pip install plotly-express nbconvert --quiet

Note: you may need to restart the kernel to use updated packages.


Figure out the SQL connection URI.

In [2]:
import os

con: str = "sqlite:///" + os.environ["HOME"] + "/results.sqlite"
con

'sqlite:////home/vscode/results.sqlite'

Read in the `experiments` table. This table contains metadata for all 'experiments' that have been run.

In [3]:
import pandas as pd

experiments: pd.DataFrame = pd.read_sql_table("experiments", con=con, index_col="id")
experiments

dataset  dataset/n  dataset/p    dataset/task dataset/group  \
id                                                                          
38vqcwus  Synclf hard      10000         50  classification        Synclf   
y6bb1hcc  Synclf hard       1000         50  classification        Synclf   
3vtr13pg  Synclf hard       1000         50  classification        Synclf   

         dataset/domain   ranker validator  \
id                                           
38vqcwus      synthetic   Boruta      k-NN   
y6bb1hcc      synthetic   Boruta      k-NN   
3vtr13pg      synthetic  ReliefF      k-NN   

                                                  local_dir  \
id                                                            
38vqcwus  /workspaces/fseval/examples/algorithm-stabilit...   
y6bb1hcc  /workspaces/fseval/examples/algorithm-stabilit...   
3vtr13pg  /workspaces/fseval/examples/algorithm-stabilit...   

                       date_created  
id                                   
38vqcwus 2022-09-21 08:22:28.965510  
y6bb1hcc 2022-09-21 08:22:53.609396  
3vtr13pg 2022-09-21 08:25:09.974370

That's looking good 🙌🏻.

Now, let's read in the `stability` table. We put data in this table by using our custom-made metric, defined in the `StabilityNogueira` class in `benchmark.py`. There, we push data to this table using `callbacks.on_table`.

In [4]:
stability: pd.DataFrame = pd.read_sql_table("stability", con=con, index_col="id")
stability

index  stability
id                        
y6bb1hcc      0   0.933546
3vtr13pg      0   1.000000

Cool. Now let's join the experiments with their actual metrics.

In [5]:
stability_experiments = stability.join(experiments)
stability_experiments

index  stability      dataset  dataset/n  dataset/p    dataset/task  \
id                                                                              
y6bb1hcc      0   0.933546  Synclf hard       1000         50  classification   
3vtr13pg      0   1.000000  Synclf hard       1000         50  classification   

         dataset/group dataset/domain   ranker validator  \
id                                                         
y6bb1hcc        Synclf      synthetic   Boruta      k-NN   
3vtr13pg        Synclf      synthetic  ReliefF      k-NN   

                                                  local_dir  \
id                                                            
y6bb1hcc  /workspaces/fseval/examples/algorithm-stabilit...   
3vtr13pg  /workspaces/fseval/examples/algorithm-stabilit...   

                       date_created  
id                                   
y6bb1hcc 2022-09-21 08:22:53.609396  
3vtr13pg 2022-09-21 08:25:09.974370

Finally, we can plot the results so we can get a better grasp of what's going on:

In [6]:
import plotly.express as px

px.bar(stability_experiments,
    x="ranker",
    y="stability"
)

We can now observe that for Boruta and ReliefF, ReliefF is the most 'stable' given this dataset, getting 100% the same features for all 10 bootstraps that were run.